In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import json
import numpy as np
import time
import random
import json

In [2]:
with open('url.json') as fopen:
    urls = json.load(fopen)

In [3]:
len(urls)

12127

In [5]:
def crawl(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "lxml")
    div = soup.find('div', {'class': 'module-content'})
    try:
        title = div.find('h1').text.strip()
    except:
        title = None
    try:
        p = div.find_all('p')
        p = [p_.text.strip() for p_ in p]
    except:
        p = None
    
    try:
        files = div.find_all('div', {'class': 'dropdown'})
        file_urls = []
        for f in files:
            a = f.find_all('a')
            for a_ in a:
                if len(a_.get('href')) > 10:
                    file_urls.append(a_.get('href'))
    except:
        file_urls = None
    
    return {
        'url': url,
        'title': title,
        'p': p,
        'file_urls': file_urls,
    }

In [11]:
from concurrent.futures import ThreadPoolExecutor, as_completed

max_worker = 10

data = []
for i in tqdm(range(0, len(urls), max_worker)):
    b = urls[i: i + max_worker]
    url = ['https://www.data.gov.my' + u for u in b]
    
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(crawl, u): u for u in url}
    
    for future in as_completed(futures):
        data.append(future.result())

100%|███████████████████████████████████████| 1213/1213 [12:23<00:00,  1.63it/s]


In [12]:
len(data)

12127

In [13]:
data[0]

{'url': 'https://www.data.gov.my/data/ms_MY/dataset/peruntukan-dan-belanja-jalan-luar-bandar-jalb',
 'title': 'PERUNTUKAN DAN BELANJA JALAN LUAR BANDAR (JALB)',
 'p': ['Data ini menerangkan tentang PERUNTUKAN DAN BELANJA JALAN LUAR BANDAR (JALB) sehingga 30 Jun 2018.',
  'Jumlah Dilihat : 37',
  'Data ini menerangkan tentang PERUNTUKAN DAN BELANJA JALAN LUAR BANDAR (JALB)...',
  'Downloads : 8'],
 'file_urls': ['/data/ms_MY/dataset/peruntukan-dan-belanja-jalan-luar-bandar-jalb/resource/9ac42a0e-0c3c-44f8-9e27-c967af68f363',
  'https://www.data.gov.my/data/dataset/6bd8d44a-c0f9-409d-b15a-8b450516e392/resource/9ac42a0e-0c3c-44f8-9e27-c967af68f363/download/peruntukan-dan-belanja-jalan-luar-bandar-jalb.xlsx']}

In [14]:
with open('mampu.json', 'w') as fopen:
    json.dump(data, fopen)